<a href="https://colab.research.google.com/github/richardtz12/Pokemon-Battle-Game/blob/master/final_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Pipeline

##1.0 Setup

### 1.1 Show me the GPU

In [1]:
!nvidia-smi

Fri May  1 08:52:10 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### 1.2 download the packages I need

In [2]:
# download 
!pip install transformers
!pip install tensorboardX
!pip install seaborn
!pip install pandas
!pip install matplotlib
!pip install transformers
!pip install captum
!pip install beautifulsoup4
!pip install nltk
!pip install google-api-python-client
!pip install Google-Search-API==1.1.14
!pip install bert-extractive-summarizer
!pip install -U spacy
!pip install neuralcoref
!python -m spacy download en_core_web_md

  Created wheel for bert-extractive-summarizer: filename=bert_extractive_summarizer-0.4.2-cp36-none-any.whl size=13711 sha256=db49e6d95b29a61ec4676deef9efa0de8ab469b5db84687ab509d4e5a12412d3
  Stored in directory: /root/.cache/pip/wheels/13/bc/30/654eb9e657177a56cba927c5a20b6cd01fb229b1ed2bf9b371
Successfully built bert-extractive-summarizer
Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)
     |████████████████████████████████| 296kB 8.9MB/s 
     |████████████████████████████████| 96.4MB 1.1MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=baf2996cd1c5d27d18d423cd997aad20ded3a2fa21fa04f1f5254d16712c606d
  Stored in directory: /tmp/pip-ephem-wheel-cache-ulsbg6bc/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [3]:
import random as r
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from textblob import TextBlob
from summarizer import Summarizer
from summarizer.coreference_handler import CoreferenceHandler

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.tokens.span.Span size changed, may indicate binary incompatibility. Expected 72 from C header, got 80 from PyObject
  return f(*args, **kwds)
100%|██████████| 40155833/40155833 [00:02<00:00, 19449716.26B/s]


### 1.3 check CUDA

In [4]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

True


## 2.0 Retrieving Passages

### 2.1 helper functions to process and return links / passages

In [0]:
from googlesearch import search
from bs4 import BeautifulSoup
import bs4
import urllib.request
import requests
import string
import nltk
from nltk.tokenize import word_tokenize
import re

"""
helper function to clean out the question
1. lower case
2. remove punctuation

NOTE: i added removing stop words thing,
but the questions get destroyed with it:
"who is the director of WHO" becomes just "director"
"""
def processQuestion(question):
    # convert to lower case, word list
    word_tokens = word_tokenize(question.lower())
    
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    word_tokens = [w.translate(table) for w in word_tokens]
            
    # return back the sentence
    return (" ").join(word for word in word_tokens)

"""
get search result links from google
"""
def getLinksFromGoogle(q, num):
  my_results_list = []
  for i in search(q,        # The query you want to run
                lang = 'en',  # The language
                num = 10,     # Number of results per page
                start = 0,    # First result to retrieve
                stop = num,  # Last result to retrieve
                pause = 3.0,  # Lapse between HTTP requests
               ):
    my_results_list.append(i)
  return my_results_list


"""
helper function that returns [String] paragraph(s) based on given URL of a wikipedia page
# TODO:-// clean out special characters  
"""
def getWikiText(link):
    webpage = urllib.request.urlopen(link).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    if len(soup.findAll("div",{"id": "toc"})) == 0:
      a = soup.find('p').text
      text_list = []
      temp = a.replace('\n', '')   # remove new line characters
      temp = re.sub('\[[0-9]{1,3}\]', '', temp)  # remove referece numbers
      
      # remove special characters, downside: all special characters are gone (some maybe used in names)
      temp =  "".join([x if ord(x) < 128 else ' ' for x in temp])
      
      # Another way to remove special characters but can't replace with a space
      # temp = item.text.encode('ascii', 'ignore').decode('utf-8')    
          
      if temp:  # some item might be empty
          text_list.append(temp)
      return text_list
    else:
      a = soup.findAll("div", {"id": "toc"})[0].find_all_previous('p', recursive=False)
      a.reverse()
      text_list = []
      for item in a:
          temp = item.text.replace('\n', '')   # remove new line characters
          temp = re.sub('\[[0-9]{1,3}\]', '', temp)  # remove referece numbers
          
          # remove special characters, downside: all special characters are gone (some maybe used in names)
          temp =  "".join([x if ord(x) < 128 else ' ' for x in temp])
          
          # Another way to remove special characters but can't replace with a space
          # temp = item.text.encode('ascii', 'ignore').decode('utf-8')    
              
          if temp:  # some item might be empty
              text_list.append(temp)
      return text_list


"""
helper function that returns [String] paragraph(s) based on given URL of a non-wikipedia page
"""
def getNonWikiText(link):
    webpage = urllib.request.Request(link, headers={'User-Agent': 'Chrome/80.0.3987.87'})
    page = urllib.request.urlopen(webpage).read()
    soup = BeautifulSoup(page, 'html.parser')
    a = soup.findAll('p')
    text_list = []
    count = 0
    num = min(len(a), 3)  # number of paragraph limits
    for item in a:
        temp = item.text.replace('\n', '')   # remove new line characters
        temp =  "".join([x if ord(x) < 128 else ' ' for x in temp])     # remove special characters
        if temp:  # some item may be empty
            text_list.append(temp)
            count += 1
            if (count >= num):
                break
    return text_list
  
def parse(string):
    """
    Parse a paragraph. Devide it into sentences and try to generate quesstions from each sentences.
    """
    
    try:
        txt = TextBlob(string)
        # Each sentence is taken from the string input and passed to genQuestion() to generate questions.
        questionArr = []
        for sentence in txt.sentences:
          newQuestion = genQuestion(sentence)
          if newQuestion is not None:
            questionArr.append(newQuestion)
        return questionArr

    except Exception as e:
        raise e



def genQuestion(line):
    """
    outputs question from the given text
    """
    

    if type(line) is str:     # If the passed variable is of type string.
        line = TextBlob(line) # Create object of type textblob.blob.TextBlob

    bucket = {}               # Create an empty dictionary


    for i,j in enumerate(line.tags):  # line.tags are the parts-of-speach in English
        if j[1] not in bucket:
            bucket[j[1]] = i  # Add all tags to the dictionary or bucket variable
    
    # if verbose:               # In verbose more print the key,values of dictionary
    
    question = ''            # Create an empty string 

    # These are the english part-of-speach tags used in this demo program.
    #.....................................................................
    # NNS     Noun, plural
    # JJ  Adjective 
    # NNP     Proper noun, singular 
    # VBG     Verb, gerund or present participle 
    # VBN     Verb, past participle 
    # VBZ     Verb, 3rd person singular present 
    # VBD     Verb, past tense 
    # IN      Preposition or subordinating conjunction 
    # PRP     Personal pronoun 
    # NN  Noun, singular or mass 
    #.....................................................................

    # Create a list of tag-combination

    l1 = ['NNP', 'VBG', 'VBZ', 'IN']
    l2 = ['NNP', 'VBG', 'VBZ']
    

    l3 = ['PRP', 'VBG', 'VBZ', 'IN']
    l4 = ['PRP', 'VBG', 'VBZ']
    l5 = ['PRP', 'VBG', 'VBD']
    l6 = ['NNP', 'VBG', 'VBD']
    l7 = ['NN', 'VBG', 'VBZ']

    l8 = ['NNP', 'VBZ', 'JJ']
    l9 = ['NNP', 'VBZ', 'NN']

    l10 = ['NNP', 'VBZ']
    l11 = ['PRP', 'VBZ']
    l12 = ['NNP', 'NN', 'IN']
    l13 = ['NN', 'VBZ']


    # With the use of conditional statements the dictionary is compared with the list created above

    
    if all(key in  bucket for key in l1): #'NNP', 'VBG', 'VBZ', 'IN' in sentence.
        question = 'What' + ' ' + line.words[bucket['VBZ']] +' '+ line.words[bucket['NNP']]+ ' '+ line.words[bucket['VBG']] + '?'

    
    elif all(key in  bucket for key in l2): #'NNP', 'VBG', 'VBZ' in sentence.
        question = 'What' + ' ' + line.words[bucket['VBZ']] +' '+ line.words[bucket['NNP']] +' '+ line.words[bucket['VBG']] + '?'

    
    elif all(key in  bucket for key in l3): #'PRP', 'VBG', 'VBZ', 'IN' in sentence.
        question = 'What' + ' ' + line.words[bucket['VBZ']] +' '+ line.words[bucket['PRP']]+ ' '+ line.words[bucket['VBG']] + '?'

    
    elif all(key in  bucket for key in l4): #'PRP', 'VBG', 'VBZ' in sentence.
        question = 'What ' + line.words[bucket['PRP']] +' '+  ' does ' + line.words[bucket['VBG']]+ ' '+  line.words[bucket['VBG']] + '?'

    elif all(key in  bucket for key in l7): #'NN', 'VBG', 'VBZ' in sentence.
        question = 'What' + ' ' + line.words[bucket['VBZ']] +' '+ line.words[bucket['NN']] +' '+ line.words[bucket['VBG']] + '?'

    elif all(key in bucket for key in l8): #'NNP', 'VBZ', 'JJ' in sentence.
        question = 'What' + ' ' + line.words[bucket['VBZ']] + ' ' + line.words[bucket['NNP']] + '?'

    elif all(key in bucket for key in l9): #'NNP', 'VBZ', 'NN' in sentence
        question = 'What' + ' ' + line.words[bucket['VBZ']] + ' ' + line.words[bucket['NNP']] + '?'

    elif all(key in bucket for key in l11): #'PRP', 'VBZ' in sentence.
        if line.words[bucket['PRP']] in ['she','he']:
            question = 'What' + ' does ' + line.words[bucket['PRP']].lower() + ' ' + line.words[bucket['VBZ']].singularize() + '?'

    elif all(key in bucket for key in l10): #'NNP', 'VBZ' in sentence.
        question = 'What' + ' does ' + line.words[bucket['NNP']] + ' ' + line.words[bucket['VBZ']].singularize() + '?'

    elif all(key in bucket for key in l13): #'NN', 'VBZ' in sentence.
        question = 'What' + ' ' + line.words[bucket['VBZ']] + ' ' + line.words[bucket['NN']] + '?'

    # When the tags are generated 's is split to ' and s. To overcome this issue.
    if 'VBZ' in bucket and line.words[bucket['VBZ']] == "’":
        question = question.replace(" ’ ","'s ")

    # Print the genetated questions as output.
    if question != '' and question is not None:
        return question


### 2.2 process question + retreive context

In [0]:
"""
main function. given a input_q, 
returns back processed input_q and a list
of paragraphs that should have the answer.
"""
def get_q_plus_paragrpaghs(input_q):
    #1. pre-process the question
    input_q = processQuestion(input_q)
    
    # 2. get at-max 10 links from wiki results
    links_wiki = getLinksFromGoogle("site:wikipedia.org " + input_q, 10)

    # get at-max 10 links from non-wiki/social media results
    links_nonwiki = getLinksFromGoogle(
            "-site:wikipedia.org -site:twitter.com -site:instagram.com -site:youtube.com -site:spotify.com " + input_q, 10)
    
    #3. get top paragraphs from each of these links
    sentences = []
    
    # extract wiki paragraphs before content list
    l = [links_wiki.index(i) for i in links_wiki if 'wikipedia.org' in i] # getting the indice of all links which are wikipedia
    if len(l) > 0: # checking if more than one such indice exists
        sentences.extend(getWikiText(links_wiki[l[0]])) # get the data from the 1st wiki link // TODO??
    
    # extract 3 paragraphs from non-wiki link
    if len(links_nonwiki) > 0: # checking if a non-wiki link exists
      sentences.extend(getNonWikiText(links_nonwiki[0])) # pulling content from the 1st non-wiki link // TODO??
    
    return input_q, sentences

### 2.2 Setup Visualizations 

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns

def show_vis(start_scores, end_scores, tokens):

  # Use plot styling from seaborn.
  sns.set(style='darkgrid')

  # Increase the plot size and font size.
  #sns.set(font_scale=1.5)
  plt.rcParams["figure.figsize"] = (16,8)

  # Pull the scores out of PyTorch Tensors and convert them to 1D numpy arrays.
  s_scores = start_scores.detach().numpy().flatten()
  e_scores = end_scores.detach().numpy().flatten()

  # We'll use the tokens as the x-axis labels. In order to do that, they all need
  # to be unique, so we'll add the token index to the end of each one.
  token_labels = []
  for (i, token) in enumerate(tokens):
    token_labels.append('{:} - {:>2}'.format(token, i))

  # Create a barplot showing the start word score for all of the tokens.
  ax = sns.barplot(x=token_labels, y=s_scores, ci=None)

  # Turn the xlabels vertical.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="center")

  # Turn on the vertical grid to help align words to scores.
  ax.grid(True)

  plt.title('Start Word Scores')

  plt.show()

  # Create a barplot showing the end word score for all of the tokens.
  ax = sns.barplot(x=token_labels, y=e_scores, ci=None)

  # Turn the xlabels vertical.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="center")

  # Turn on the vertical grid to help align words to scores.
  ax.grid(True)

  plt.title('End Word Scores')

  plt.show()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### 2.3 get predictions using pre-trained ALBERT model on SQuAD dataset

training code [here](https://colab.research.google.com/drive/1P4lUkvxCzv4m8fmxHU3wGwMYYgFA7f0w#scrollTo=frTeTcy4WdbY) on `ALBERT finetuning.ipynb`

In [8]:
# get the model_ouput (all checkpoints) from GCS bucket
!wget https://storage.googleapis.com/ica_2020/model_output.zip
!unzip model_output.zip

--2020-05-01 08:53:40--  https://storage.googleapis.com/ica_2020/model_output.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.126.128, 2a00:1450:4013:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2027831147 (1.9G) [application/zip]
Saving to: ‘model_output.zip.1’

model_output.zip.1  100%[===================>]   1.89G  39.3MB/s    in 27s     

2020-05-01 08:54:07 (72.3 MB/s) - ‘model_output.zip.1’ saved [2027831147/2027831147]

Archive:  model_output.zip
replace content/model_output/checkpoint-4000/optimizer.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: All
  inflating: content/model_output/checkpoint-4000/optimizer.pt  
  inflating: content/model_output/checkpoint-4000/scheduler.pt  
  inflating: content/model_output/checkpoint-4000/tokenizer_config.json  
  inflating: content/model_output/checkpoint-4000/pytorch_model.bin  
 extracting: content/model_

In [0]:
import os
import torch
import time
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import (
    AlbertConfig,
    AlbertForQuestionAnswering,
    AlbertTokenizer,
    squad_convert_examples_to_features
)

from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample

from transformers.data.metrics.squad_metrics import compute_predictions_logits

# READER NOTE: Set this flag to use own model, or use pretrained model in the Hugging Face repository
use_own_model = True

if use_own_model:
  model_name_or_path = "content/model_output"
else:
  model_name_or_path = "ktrapeznikov/albert-xlarge-v2-squad-v2"

output_dir = ""

# Config
n_best_size = 3
max_answer_length = 30
do_lower_case = True
null_score_diff_threshold = 0.0

def to_list(tensor):
    return tensor.detach().cpu().tolist()

# Setup model
config_class, model_class, tokenizer_class = (
    AlbertConfig, AlbertForQuestionAnswering, AlbertTokenizer)
config = config_class.from_pretrained(model_name_or_path)
tokenizer = tokenizer_class.from_pretrained(
    model_name_or_path, do_lower_case=True)
model = model_class.from_pretrained(model_name_or_path, config=config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

processor = SquadV2Processor()

def run_prediction(question_texts, context_text):
    """Setup function to compute predictions"""
    examples = []

    for i, question_text in enumerate(question_texts):
        example = SquadExample(
            qas_id=str(i),
            question_text=question_text,
            context_text=context_text,
            answer_text=None,
            start_position_character=None,
            title="Predict",
            is_impossible=False,
            answers=None,
        )

        examples.append(example)

    features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=384,
        doc_stride=128,
        max_query_length=64,
        is_training=False,
        return_dataset="pt",
        threads=1,
    )

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

    all_results = []

    for batch in eval_dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs)

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)

                output = [to_list(output[i]) for output in outputs]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
                all_results.append(result)

    output_prediction_file = "predictions.json"
    output_nbest_file = "nbest_predictions.json"
    output_null_log_odds_file = "null_predictions.json"

    predictions = compute_predictions_logits(
        examples,
        features,
        all_results,
        n_best_size,
        max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,  # verbose_logging
        True,  # version_2_with_negative
        null_score_diff_threshold,
        tokenizer,
    )

    return predictions

### 2.4 examples

In [0]:
context = "The Georgia Institute of Technology, commonly referred to as Georgia Tech or, in the state of Georgia, as Tech,[8] is a public research university and institute of technology in Atlanta, Georgia.[9] It is part of the University System of Georgia and has satellite campuses in Savannah, Georgia; Metz, France; Athlone, Ireland; Shenzhen, China; and Singapore. The school was founded in 1885 as the Georgia School of Technology as part of Reconstruction plans to build an industrial economy in the post-Civil War Southern United States. Initially, it offered only a degree in mechanical engineering. By 1901, its curriculum had expanded to include electrical, civil, and chemical engineering. In 1948, the school changed its name to reflect its evolution from a trade school to a larger and more capable technical institute and research universi"
questions = ["where is georgia tech?"]

# Run method
predictions = run_prediction(questions, context)

# Print results
for key in predictions.keys():
  print(predictions[key])

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 6482.70it/s]

Atlanta, Georgia.[9]


In [0]:
input_q, paras = get_q_plus_paragrpaghs("Who is the secretary general of united nations?")
paras = ''.join(paras)
if len(paras) > 400:
  paras = paras[:400]
predictions = run_prediction([input_q], paras)

# Print results
for key in predictions.keys():
  print(predictions[key])

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3182.32it/s]

The secretary-general serves as the chief administrative officer of the United Nations.


In [0]:
input_q, paras = get_q_plus_paragrpaghs("where is georgia tech")
paras = ''.join(paras)
if len(paras) > 400:
  paras = paras[:400]
predictions = run_prediction([input_q], paras)

# Print results
for key in predictions.keys():
  print(predictions[key])

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2353.71it/s]


is a public research university and institute of technology in Atlanta, Georgia.


In [0]:
input_q, paras = get_q_plus_paragrpaghs("what is stack overflow")
paras = ''.join(paras)
if len(paras) > 400:
  paras = paras[:400]
predictions = run_prediction([input_q], paras)

# Print results
for key in predictions.keys():
  print(predictions[key])

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2261.08it/s]


if the call stack pointer exceeds the stack bound


In [0]:
input_q, paras = get_q_plus_paragrpaghs("what is github")
paras = ''.join(paras)
if len(paras) > 400:
  paras = paras[:400]
predictions = run_prediction([input_q], paras)

# Print results
for key in predictions.keys():
  print(predictions[key])

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 368.28it/s]


GitHub, Inc. is a United States-based global company that provides hosting for software development version control using Git.


In [0]:
input_q, paras = get_q_plus_paragrpaghs("who is the founder of slack")
paras = ''.join(paras)
if len(paras) > 400:
  paras = paras[:400]
predictions = run_prediction([input_q], paras)

# Print results
for key in predictions.keys():
  print(predictions[key])

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2132.34it/s]


Daniel Stewart Butterfield


In [0]:
input_q, paras = get_q_plus_paragrpaghs("who is the founder of facebook")
paras = ''.join(paras)
if len(paras) > 400:
  paras = paras[:400]
predictions = run_prediction([input_q], paras)

# Print results
for key in predictions.keys():
  print(predictions[key])

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3603.35it/s]


Mark Elliot Zuckerberg


In [0]:
input_q, paras = get_q_plus_paragrpaghs("who made python")
paras = ''.join(paras)
if len(paras) > 400:
  paras = paras[:400]
predictions = run_prediction([input_q], paras)

# Print results
for key in predictions.keys():
  print(predictions[key])

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2024.28it/s]


Guido van Rossum


In [0]:
input_q, paras = get_q_plus_paragrpaghs("who is george p burdell")
paras = ''.join(paras)
if len(paras) > 400:
  paras = paras[:400]
predictions = run_prediction([input_q], paras)

# Print results
for key in predictions.keys():
  print(predictions[key])

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 6875.91it/s]

George P. Burdell is a fictitious student officially enrolled at Georgia Tech in 1927 as a practical joke.


### 2.5 Interactive example

In [10]:
while True:
    inputt = input("YOU: ")
    if inputt == "END":
        break
    else:
        lastPeriod = 0
        for i in range(len(inputt)):
          if inputt[i] == "." or inputt[i] == "!":
            lastPeriod = i
        inputt = inputt[lastPeriod:]
        input_q, paras = get_q_plus_paragrpaghs(inputt)
        paras = ''.join(paras)
        if len(paras) > 400:
          paras = paras[:400]
        sumModel = Summarizer() 
        summ = sumModel(paras, ratio=1/2)
        predictions = run_prediction([input_q], paras)
        for key in predictions.keys():
          if predictions[key] is None or predictions[key] == "":
            print("Sorry I don't know the answer to that question! Maybe try asking something else?")
          else:
            out = "ICA: {}".format(predictions[key])
            questionArr = parse(summ)
            if len(questionArr) > 0:
              randNum = r.randint(0, len(questionArr) - 1)
              if input_q.lower().strip() == questionArr[randNum][:-1].lower().strip():
                print (out + ". " + "What do you think?")
              else:
                print (out + ". " + questionArr[randNum])
            else:
              print (out + ". ")

#what is a coronavirus?
#what is COVID-19?
#what is death?
#what is the meaning of life?

YOU: What is death?


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 418.80it/s]


ICA: the cessation of all biological functions that sustain a living organism. What do you think?
YOU: I think it's a rite of passage from this life to the afterlife. Where do people go after they die?


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 4928.68it/s]


ICA: the physical body. What lives afterlife According?
YOU: I think ghosts live in the afterlife. Are ghosts real?


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2225.09it/s]


ICA: descriptions of ghosts vary widely from an invisible presence to translucent or barely visible wispy shapes, to realistic, lifelike forms.. What is folklore?


KeyboardInterrupt: ignored

# 3.0 Generate Questions